### Exporting the processed intersections, bus stops and focus points to the VED files.

In [1]:
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import folium
import csv

Read three CSV files: intersections, focus points and bus stops.

In [ ]:
intersections = pd.read_csv('../data/processed/joined_coords_intersections.csv').to_numpy()
focus_points = pd.read_csv('../data/processed/joined_layer_coords_focus_points.csv')
# combine different focus points in the 'highway' column
focus_points['highway'] = focus_points.bfill(1)['highway']
focus_points = focus_points.to_numpy()
busstops = pd.read_csv('../data/processed/joined_coords_bus_stops.csv').to_numpy()

Create dictionaries where keys are the latitude/longitude coordinates.

In [3]:
intersections_dict = {(intersections[i,0], intersections[i,1]) : 1 for i in range(len(intersections))}
busstops_dict = {(busstops[i,0], busstops[i,1]) : 1 for i in range(len(busstops))}
focus_points_dict = {(focus_points[i,0], focus_points[i,1]) : focus_points[i,4] for i in range(len(focus_points))}

Let's visualize some of the intersections. Folium map is slow if we try to plot all of them. Therefore, we will plot the first 1000 intersections.

In [4]:
def plot_map(latlon):
    tiles = "cartodbpositron"
    map = folium.Map(prefer_canvas=True, tiles=tiles)
    t = folium.TileLayer(tiles).add_to(map)
    lats = [point[0] for point in latlon]
    lons = [point[1] for point in latlon]
    min_lat, max_lat = min(lats), max(lats)
    min_lon, max_lon = min(lons), max(lons)
    map.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])
    for point in latlon:
        folium.CircleMarker(
        location=[point[0], point[1]], radius=2, fill=True, color='red',
        popup='').add_to(map)
    return map

plot_map(list(intersections_dict.keys())[0:1000])

In [4]:
data_path = "../data/ved-speed-limits/"
files = [os.path.join(data_path, file) for file in tqdm(os.listdir(data_path)) if file.endswith(".csv")]
files.sort()

  0%|          | 0/54 [00:00<?, ?it/s]

In [9]:
columns = ['DayNum', 'VehId', 'Trip', 'Timestamp(ms)', 'Latitude[deg]', 
           'Longitude[deg]', 'Vehicle Speed[km/h]', 'MAF[g/sec]', 
           'Engine RPM[RPM]', 'Absolute Load[%]', 'OAT[DegC]', 'Fuel Rate[L/hr]', 
           'Air Conditioning Power[kW]', 'Air Conditioning Power[Watts]', 
           'Heater Power[Watts]', 'HV Battery Current[A]', 'HV Battery SOC[%]', 
           'HV Battery Voltage[V]', 'Short Term Fuel Trim Bank 1[%]', 
           'Short Term Fuel Trim Bank 2[%]', 'Long Term Fuel Trim Bank 1[%]', 
           'Long Term Fuel Trim Bank 2[%]', 'Matchted Latitude[deg]', 'Matched Longitude[deg]', 
           'Match Type', 'Class of Speed Limit', 'Speed Limit[km/h]', 
           'Speed Limit with Direction[km/h]', 'Intersection', 'Focus Points', 'Bus Stops']

def get_save_path(filename):
    return f"../data/ved_with_intersections_focus_points/{filename[len(data_path)-1:]}"

def export_data(files, columns):
    
    for file in tqdm(files):
        
        ved_file = pd.read_csv(file, dtype={'Speed Limit[km/h]': 'str'}).to_numpy()
        N = len(ved_file)
        ved_file_sp = np.c_[ved_file, np.zeros(N), np.zeros(N), np.empty(N, dtype='<U10')]
        for f in ved_file_sp:
            inters = intersections_dict.get((float(f[22]), float(f[23])), np.nan)
            crossing = focus_points_dict.get((float(f[22]), float(f[23])), np.nan)
            busstop = busstops_dict.get((float(f[22]), float(f[23])), np.nan)
            f[28], f[29], f[30] = inters, busstop, crossing
        
        with open(get_save_path(file), "w", newline='') as f2:
            write = csv.writer(f2) 
            write.writerow(columns) 
            write.writerows(ved_file_sp)
            f2.close()

In [12]:
export_data(files, columns)

  0%|          | 0/54 [00:00<?, ?it/s]

### Combining all data by reading the files into DataFrames, merging relevant columns and writing the final 
### version of VED to the directory `ved-final`

In [16]:
path_grad = "../data/final-data-with-gradient/"
files_grad = [os.path.join(path_grad, file) for file in tqdm(os.listdir(path_grad)) if file.endswith(".csv")]
files_grad.sort()

path_inters = "../data/ved_with_intersections_focus_points/"
files_intrs = [os.path.join(path_inters, file) for file in tqdm(os.listdir(path_inters)) if file.endswith(".csv")]
files_intrs.sort()

  0%|          | 0/54 [00:00<?, ?it/s]

  0%|          | 0/54 [00:00<?, ?it/s]

In [50]:
columns_inters = list(pd.read_csv(files_intrs[0]).columns)
columns_grad = list(pd.read_csv(files_grad[0]).columns)

def get_save_path(file):
    return f"../data/ved-final/{file[len(path_inters):]}"

for i in tqdm(range(len(files_intrs))):
    df_grad = pd.read_csv(files_grad[i])
    # Take the last 9 columns of df_intrs and merge them with df_grad
    df_intrs = pd.read_csv(files_intrs[i], dtype={'Speed Limit[km/h]': 'str'})[columns_inters[-9:]]
    # Delete old columns for speed limit, class of speed limit and focus points
    df_grad = df_grad.drop(columns_grad[-3:], 1)
    df_grad = df_grad.merge(df_intrs, left_index=True, right_index=True)
    df_grad.to_csv(get_save_path(files_intrs[i]), na_rep=np.nan, index=False)


  0%|          | 0/54 [00:00<?, ?it/s]